# Hands-on: Hadronization - $e^+e^-$ Analysis

In this hands-on session we want to analyze the output of X-SCAPE framework simulations with $e^+e^-$ collision events. We have generated data for collisions at $\sqrt{s}=91.2$ GeV similar to the experimental runs of the ALEP experiment.

In [1]:
# prerequisites: plots and packages
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import eig
import fastjet as fj
import sparkx.Jetscape as spxJ

from JetAnalysis import JetAnalysis

# put some settings for the output
width = 0.05
plotMarkerSize = 8
labelfontsize = 15
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [6., 4.5]
mpl.rcParams['lines.linewidth'] = 2
mpl.rcParams['xtick.top'] = True
mpl.rcParams['xtick.labelsize'] = 13
mpl.rcParams['xtick.major.width'] = 1.0
mpl.rcParams['xtick.minor.width'] = 0.8
mpl.rcParams['xtick.minor.visible'] = True
mpl.rcParams['xtick.direction'] = "in"
mpl.rcParams['ytick.right'] = True
mpl.rcParams['ytick.labelsize'] = 13
mpl.rcParams['ytick.major.width'] = 1.0
mpl.rcParams['ytick.minor.width'] = 0.8
mpl.rcParams['ytick.minor.visible'] = True
mpl.rcParams['ytick.direction'] = "in"
mpl.rcParams['legend.fontsize'] = 13
mpl.rcParams['legend.numpoints'] = 1
mpl.rcParams['font.size'] = 13
mpl.rcParams['savefig.format'] = "pdf"

In [3]:
# load the data from the directory
PATH_HADRONIZATION_DATA = '../../../X-SCAPE/build/hadronization_results/hadrons_epem_final_state_hadrons.dat'

simulation_data = spxJ.Jetscape(PATH_HADRONIZATION_DATA,events=(0,5000))
full_particle_list = simulation_data.particle_objects_list()


## Sphericity Tensor, Sphericity and Aplanarity

We construct the second rank sphericity tensor
$$ \theta_{ij} \equiv \frac{1}{Q_p}\left| \sum\limits_a \frac{\vec{p}_{a,i}\vec{p}_{a,j}}{|\vec{p}_{a}|}\right|, $$
where the index $a$ labels the final state particles and $Q_p$ is sum of the modulus of the 3-momenta
$$Q_p\equiv \sum\limits_j |\vec{p}_j|.$$

The eigenvalues of this tensor have a geometrical meaning. First, they obey $\sum_{i=1}^3\lambda_i=3$.
If the event is a perfectly back-to-back jet, then two of the eigenvalues are zero.
If the event is a three-jet event, then just one of the eigenvalues is zero.
In the case of a spherical momentum distribution the eigenvalues are $\lambda_1=\lambda_2=\lambda_3=1/3$ [[Donoghue et. al.]](https://journals.aps.org/prd/abstract/10.1103/PhysRevD.20.2759).

The event shape is then usually defined in terms of sphericity
$$ S = \frac{3}{2}(\lambda_1 + \lambda_2) $$
and aplanarity
$$ A = \frac{3}{2}\lambda_1, $$
where the order $\lambda_3 \geq\lambda_2\geq\lambda_1$ is assumed [[Renard]](https://books.google.de/books?id=tsfhNxb8kX0C&pg=PA147&lpg=PA147&dq=electron+positron+collisions+sphericity&source=bl&ots=vIToRKbBJP&sig=ACfU3U1UwfaVbhhq5261BoMMdXp0C1uRtQ&hl=de&sa=X&ved=2ahUKEwiYr_jJ--T8AhXOwAIHHSXWDKM4ChDoAXoECCEQAw#v=onepage&q=electron%20positron%20collisions%20sphericity&f=false).

In [ ]:
def extract_momenta(event_hadrons):
    # 3d array (next deeper level contains events, each event contains some 3-momentum vectors of its hadrons)
    momentum_vectors = []
    energies = [] 
    # loop over the events and create the momentum vectors
    for ev in range(len(event_hadrons)):
        event_momenta = []
        for h in range(len(event_hadrons[ev])):
            event_momenta.append(np.array([event_hadrons[ev][h].px,event_hadrons[ev][h].py,event_hadrons[ev][h].pz]))
        if len(event_hadrons[ev]) > 0: # neglects the empty parts with events deleted by detector cuts
            momentum_vectors.extend([event_momenta])
    return momentum_vectors 

def Qp(p):
    norms = np.linalg.norm(p, axis=1)
    Qp = np.sum(norms)
    return Qp

def momentum_tensor(momenta):
    normalization = Qp(momenta)
    tensor = np.zeros((3, 3))
    
    for alpha in range(3):
        for beta in range(3):
            tens_alpha_beta = 0.0
            for i in range(len(momenta)):
                norm_i = np.linalg.norm(momenta[i])
                tens_alpha_beta += momenta[i][alpha] * momenta[i][beta] / norm_i
            tens_alpha_beta = np.abs(tens_alpha_beta) / normalization
            tensor[alpha][beta] = tens_alpha_beta
    
    return tensor

def compute_sphericity(eigenvalues):
    return (3./2.)*(eigenvalues[1]+eigenvalues[2])

def compute_aplanarity(eigenvalues):
    return (3./2.)*eigenvalues[0]

def compute_sphericity_tensor_quantities(momenta):
    # compute the sphericity and the aplanarity for all events
    events_sphericity = []
    events_aplanarity = []
    events_eigenvalues = []
    events_eigenvectors = []

    for ev in range(len(momenta)):
        mom_tens = momentum_tensor(momenta[ev])
        evalue, evec = eig(mom_tens)
        # sort the eigenvalues and eigenvectors in decreasing order of the eigenvalues
        idx = evalue.argsort()[::-1]   
        eigenValues = evalue[idx]
        eigenVectors = evec[:,idx]
    
        events_eigenvalues.extend([eigenValues])
        events_eigenvectors.extend([eigenVectors])
        events_sphericity.append(compute_sphericity(eigenValues))
        events_aplanarity.append(compute_aplanarity(eigenValues))
    return events_sphericity, events_aplanarity, events_eigenvalues, events_eigenvectors

momenta = extract_momenta(full_particle_list)
sphericity, aplanarity, eigenvalues, eigenvectors = compute_sphericity_tensor_quantities(momenta)

### The "Q-plot"

Now we generate a $\lambda$ plot to show the overall event geometry in the $e^+ e^-$-collisions.
Therefore we plot $\frac{\sqrt{3}}{2}(\lambda_2-\lambda_3)$ vs. $S$.
Events in the lower left corner are the two jet events. In the upper region along the line defined by $A$ there are planar events and in the lower right corner one can find spherical events.

In this older [reference](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8141838/) there is also a nice Q-plot.

In [ ]:
def plot_Qplot(evals,spher):
    y_axis_quantity = []
    for ev in range(len(evals)):
        y_axis_quantity.append(np.sqrt(3)*(evals[ev][1]-evals[ev][2])/2.)

    def Qplot_triangle(S):
        return np.piecewise(S, [S < 3./4., S >= 3./4.], [lambda S: np.sqrt(3.)/3. * S, lambda S: -np.sqrt(3.)*(S-(3./4.))+(np.sqrt(3)/4.)])

    xrange = np.arange(0,1,0.0001)
    yrange = Qplot_triangle(xrange)

    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.scatter(spher,y_axis_quantity,color='g',s=1)
    ax.plot(xrange,yrange,color='k')
    ax.set_xlabel(r"$S$")
    ax.set_ylabel(r"$\sqrt{3}(\lambda_2-\lambda_1)/2$")
    ax.set_aspect('equal')
    plt.text(0.28, 0.18, r'$\lambda_1=0$', dict(size=13), rotation = 30, rotation_mode = 'anchor')
    plt.text(0.88, 0.24, r'$\lambda_2=\lambda_3$', dict(size=13), rotation = -60, rotation_mode = 'anchor')
    plt.text(0.83, 0.02, r'$\lambda_1=\lambda_2$', dict(size=13), rotation = 0, rotation_mode = 'anchor')

    plt.xlim(0,None)
    plt.ylim(0,None)
    plt.tight_layout()
    plt.savefig('./Q_plot.pdf')

plot_Qplot(eigenvalues,sphericity)

### Multiplicity distribution

The next observable we want to analyze is the multiplicity of the events. We plot the probability to emmit $n$ charged hadrons $p(n) = \frac{1}{N_\mathrm{ev}}\frac{\mathrm{d}N_\mathrm{ch}}{\mathrm{d}n}$.

The cuts we will apply here are arbitrary. They do not coincide with the ones from the ALEPH collaboration. In their case there are also event cuts which depend on the sphericity axis, which we are not computing here (computationally more expensive).

In [4]:
simulation_data = spxJ.Jetscape(PATH_HADRONIZATION_DATA)
cut_particle_list = simulation_data.charged_particles().pt_cut((0.2,None)).pseudorapidity_cut((-2.6,2.6)).particle_objects_list()

def compute_multiplicities(hadrons):
    multiplicity = [len(event) for event in hadrons if len(event) > 0]
    return multiplicity, len(multiplicity)

multiplicity, N_events = compute_multiplicities(cut_particle_list)

# fill histogram 
val_hist, q_bin = np.histogram(multiplicity, bins=range(1,60))

# statistical errors y-axis
err_hist = np.sqrt(val_hist)

# bin width x-axis
dq = (q_bin[1:]-q_bin[:-1])

# bin center x-axis
q_bin = q_bin[0:-1] + 0.5*dq

# construct y-axis spectrum (scaled) 
dN_dq = val_hist/N_events/dq

# error bars y-axis 
err_dN_dq = err_hist/N_events/dq
    
# generate plot
fig = plt.figure()
plt.errorbar(q_bin, dN_dq, fmt='s', xerr=0.5*dq, yerr=err_dN_dq, color='g')

#axes setting
plt.yscale('log')
plt.xlabel(r"$n$")
plt.ylabel(r"$p(n)$")
plt.savefig('./epem_multiplicity.pdf')

<IPython.core.display.Javascript object>

## Jet analysis

In the following part we will analyze the jet observables by using the FastJet code with the anti-$k_\mathrm{T}$ algorithm in its $e^+e^-$ variant.
The considered resolution parameter is $R_0 = 0.4$, as it is widely chosen for other collision systems.
The distance metric is
$$ d_{i,j}\equiv \min(E_i^{-2},E_j^{-2})\frac{1-\cos\theta_{ij}}{1-\cos R_0} $$
$$ d_{iB} \equiv E_i^{-2}, $$
with $d_{ij}$ the distance between two pseudojets $i$ and $j$, $E_i$ the energy and $\theta_{ij}$ the opening angle.

In [12]:
# take the cut_particle_list and analyze it in fastjet
jets = JetAnalysis()
jets.perform_jet_finding(cut_particle_list, jet_R=0.4, jet_eta_range=(-2.,2.), jet_pt_range=(10.,None),output_filename="epem_jets.dat")

jets.read_jet_data("./epem_jets.dat")

jet_data = jets.get_jets()
associated_data = jets.get_associated_particles()

jet_energies = [jet[6] for jet in jet_data]


jet definition is: Longitudinally invariant anti-kt algorithm with R = 0.4 and E scheme recombination
jet selector is: -2 <= eta <= 2
[29.87611, 20.176319999999997, 15.593894, 38.1203, 41.13374, 27.019482, 16.13587, 17.65175, 13.27814, 30.242, 11.49767, 22.5902, 37.57805, 18.23081, 28.146430000000002, 23.01193, 32.43865, 25.757399999999997, 16.77129, 32.96949000000001, 26.362920000000003, 24.858951, 38.65943, 15.761777, 16.369670000000003, 17.909502, 31.766219999999997, 25.121148, 16.12175, 35.56768, 18.95181, 39.13511, 32.18217, 30.485341, 15.468705000000003, 18.38091, 25.843517, 31.60315, 36.37168, 33.05083, 27.443309999999997, 20.3289, 43.09329, 33.088264, 30.54895, 22.07762, 26.852719999999998, 29.8054, 36.23801, 35.87043, 27.41749, 40.771550000000005, 18.86778, 26.716, 32.601760000000006, 34.303149999999995, 33.4602, 32.894380000000005, 36.762418, 26.117009999999997, 26.200353, 26.572329999999997, 18.497816, 11.021930000000001, 26.814230000000002, 23.770106, 25.419239999999995, 26

### Jet Energy Spectra

The jet energy spectrum is defined by 
$$\frac{1}{N_{\mathrm{ev}}} \frac{\mathrm{d}N_{\mathrm{jet}}}{\mathrm{d}E_{\mathrm{jet}}}.$$
$N_{\mathrm{ev}}$ is the number of events with hard scatterings simulated and $N_{\mathrm{jet}}$ is the total number of jets.

In [17]:
# bin settings
E_min = 10.
E_max = 65.
E_bins = np.arange(E_min, E_max, 1.0)

# Fill Histogram
n, E = np.histogram(jet_energies, bins=E_bins)

# Statistical Errors
err_n = np.sqrt(n)

# bin width
dE = (E[1:]-E[:-1])
# bin center
E = E[0:-1] + 0.5*dE

# Jet Spectrum 
dn_dE = n/N_events/dE

# Errors 
err_dn_dE = err_n/N_events/dE

# Generate Plots
fig = plt.figure()

plt.errorbar(E, dn_dE, fmt='s', label="decays",
             xerr=0.5*dE, yerr=err_dn_dE, color='g')

#axes setting
plt.yscale('log')
plt.legend(loc=0)
plt.xlabel(r"$E_\mathrm{jet}$ [GeV]")
plt.ylabel(r"$(1/N_{\mathrm{ev}})\mathrm{d}N_{\mathrm{jet}}/\mathrm{d}E_\mathrm{jet}$")
plt.xlim(E_min,E_max)

# save plots
plt.tight_layout()
plt.savefig('./n_jet.pdf')

<IPython.core.display.Javascript object>